In [1]:
gamma = 1.3
R = 287

287

In [2]:
f = 0.003 # Friction Factor
D = 0.3 # m (Diameter)
Lt = 10.8

10.8

In [3]:
function L_star(M)  # Fanno Flow Relation
    return ((1 - M^2)/(gamma * M^2) + ((gamma + 1) / (2 * gamma))* log((gamma + 1) * M^2 / (2 + (gamma - 1) * M^2))) * D / (4 * f)
end

L_star (generic function with 1 method)

In [4]:
function M2(M1) # Normal Shock Mach Relation
    return ((1 + ((gamma - 1) / 2) * M1^2)/(gamma * M1^2 - (gamma - 1) / 2))^0.5
end

M2 (generic function with 1 method)

In [5]:
using JuMP
import Ipopt

In [6]:
model = Model(Ipopt.Optimizer) # Using Non Linear solver

A JuMP Model
├ solver: Ipopt
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [7]:
@variable(model, Mx, lower_bound = 1)

Mx

In [8]:
@variable(model, My, lower_bound = 0)

My

In [9]:
@variable(model, La, lower_bound = 0)

La

In [10]:
@variable(model, Lb, lower_bound = 0)

Lb

In [11]:
@constraint(model, La == L_star(2) - L_star(Mx))  # Length before shock


La - (8.931934142052128 - (((((-Mx² + 1) / (1.3 Mx²)) + (0.8846153846153845 * log((2.3 Mx²) / (0.30000000000000004 Mx² + 2)))) * 0.3) / 0.012)) == 0

In [12]:
@constraint(model, Lb == L_star(My))              # Length after shock

Lb - (((((-My² + 1) / (1.3 My²)) + (0.8846153846153845 * log((2.3 My²) / (0.30000000000000004 My² + 2)))) * 0.3) / 0.012) == 0

In [13]:
@constraint(model, My == M2(Mx))

My - (((0.15000000000000002 Mx² + 1) / (1.3 Mx² - 0.15000000000000002)) ^ 0.5) == 0

In [16]:
@objective(model,Min ,(La+Lb-Lt)^2) # #la+lb-Lt=0. (abs function takes too much time)

La² + 2 Lb*La + Lb² - 21.6 La - 21.6 Lb + 116.64000000000001

In [17]:
# Solve the model
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        4
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality c

In [18]:
Mx_val = value(Mx)


1.4691532700054162

In [19]:
My_val = value(My)

0.7059861916028758

In [20]:
La_val = value(La)


5.350026368902088

In [21]:
Lb_val = value(Lb)

5.449973631535962